In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 23689276
paper_name = 'bowie_fyles_2013' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [20]:
original_data1 = pd.read_excel('raw_data/c3ob40593a.xlsx', sheet_name='Table S1', skiprows=2)
original_data2 = pd.read_excel('raw_data/c3ob40593a.xlsx', sheet_name='Table S2', skiprows=2)

In [21]:
print('Original data dimensions: %d x %d' % (original_data1.shape))

Original data dimensions: 6144 x 13


In [22]:
original_data1['Orf'] = original_data1['Orf'].astype(str)
original_data2['Orf'] = original_data2['Orf'].astype(str)

In [23]:
# Eliminate all white spaces & capitalize
original_data1['Orf'] = clean_orf(original_data1['Orf'])
original_data2['Orf'] = clean_orf(original_data2['Orf'])

In [24]:
# Translate to ORFs 
original_data1['Orf'] = translate_sc(original_data1['Orf'], to='orf')
original_data2['Orf'] = translate_sc(original_data2['Orf'], to='orf')

In [25]:
# Make sure everything translated ok
t = looks_like_orf(original_data1['Orf'])
print(original_data1.loc[~t,])

             P   R   C        Orf       Gene  Info Ignore?  SL  SR  Hits  \
index_input                                                                
5516         4  28  44  UNDEFINED  undefined  INFO       N   3   0     3   
5522         4  15  28  UNDEFINED  undefined  INFO       N   2   0     2   
5523         4  23  47  UNDEFINED  undefined  INFO       N   2   0     2   
5524         4  27  44  UNDEFINED  undefined  INFO       N   2   0     2   
5525         4  27  48  UNDEFINED  undefined  INFO       N   2   0     2   
...         ..  ..  ..        ...        ...   ...     ...  ..  ..   ...   
6139         4  32  36  UNDEFINED  undefined  INFO       N   0   0     0   
6140         4  32  39  UNDEFINED  undefined  INFO       N   0   0     0   
6141         4  32  40  UNDEFINED  undefined  INFO       N   0   0     0   
6142         4  32  47  UNDEFINED  undefined  INFO       N   0   0     0   
6143         4  32  48  UNDEFINED  undefined  INFO       N   0   0     0   

           

In [26]:
original_data1 = original_data1.loc[t,:]

In [27]:
# Make sure everything translated ok
t = looks_like_orf(original_data2['Orf'])
print(original_data2.loc[~t,])

             P   R   C        Orf       Gene  Info Ignore?  SL  SR  Hits  \
index_input                                                                
5693         1   1  18  UNDEFINED  undefined  INFO       N   1   0     0   
5704         4  23  28  UNDEFINED  undefined  INFO       N   1   0     0   
5705         4  24   4  UNDEFINED  undefined  INFO       N   1   0     0   
5706         4  24  28  UNDEFINED  undefined  INFO       N   1   0     0   
5707         4  29  38  UNDEFINED  undefined  INFO       N   1   0     0   
...         ..  ..  ..        ...        ...   ...     ...  ..  ..   ...   
6137         4  32  40  UNDEFINED  undefined  INFO       N   0   0     0   
6140         4  32  43  UNDEFINED  undefined  INFO       N   0   0     0   
6141         4  32  44  UNDEFINED  undefined  INFO       N   0   0     0   
6142         4  32  47  UNDEFINED  undefined  INFO       N   0   0     0   
6143         4  32  48  UNDEFINED  undefined  INFO       N   0   0     0   

           

In [28]:
original_data2 = original_data2.loc[t,:]

In [29]:
original_data1.set_index('Orf', inplace=True)
original_data2.set_index('Orf', inplace=True)

In [30]:
original_data1.index.name='orf'
original_data2.index.name='orf'

In [32]:
original_data1['data'] = pd.to_numeric(original_data1['Ratio'], errors='coerce')
original_data2['data'] = pd.to_numeric(original_data2['Ratio'], errors='coerce')

In [33]:
original_data1 = original_data1[['data']].copy()
original_data2 = original_data2[['data']].copy()

In [34]:
original_data1 = original_data1.groupby(original_data1.index).mean()
original_data2 = original_data2.groupby(original_data2.index).mean()

In [35]:
original_data = original_data1.join(original_data2, how='outer', lsuffix='_1', rsuffix='_2')

In [36]:
original_data.head()

,data_1,data_2
orf,,
YAL002W,1.113614,1.039779
YAL004W,0.967410,0.949484
YAL005C,1.075862,0.940879
YAL007C,1.069896,1.032866
YAL008W,0.999591,1.003353


In [37]:
original_data.shape

(4765, 2)

# Prepare the final dataset

In [38]:
data = original_data.copy()

In [39]:
dataset_ids = [16557, 16558]

In [40]:
datasets = datasets.reindex(index=dataset_ids)

In [41]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [42]:
data.head()

dataset_id,16557,16558
data_type,value,value
orf,,
YAL002W,1.113614,1.039779
YAL004W,0.967410,0.949484
YAL005C,1.075862,0.940879
YAL007C,1.069896,1.032866
YAL008W,0.999591,1.003353


## Subset to the genes currently in SGD

In [43]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [44]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16557,16558
,data_type,value,value
gene_id,orf,,
2,YAL002W,1.113614,1.039779
1863,YAL004W,0.967410,0.949484
4,YAL005C,1.075862,0.940879
5,YAL007C,1.069896,1.032866
6,YAL008W,0.999591,1.003353


# Normalize

In [45]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [46]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [47]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16557,16558,16557,16558
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
2,YAL002W,1.113614,1.039779,0.163926,0.539638
1863,YAL004W,0.967410,0.949484,0.006026,-0.785757
4,YAL005C,1.075862,0.940879,0.123154,-0.912058
5,YAL007C,1.069896,1.032866,0.116712,0.438166
6,YAL008W,0.999591,1.003353,0.040782,0.004967


# Print out

In [48]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [49]:
from IO.save_data_to_db3 import *

In [50]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 23689276...
Inserting the new data...


100%|██████████| 2/2 [00:15<00:00,  7.60s/it]

Updating the data_modified_on field...
